In [124]:
#  hier wurde eine geeignete einhüllende Funktion gesucht, kann ignoriert werden
import numpy as np
import matplotlib.pyplot as plt
(3+np.cos(1/2)**2)**2/np.sin(1/2)**4/2**2
1/0.13**3
b=0.7
x = np.linspace(1,np.pi)
y = 1/np.sin(x/2)**4-b/np.sin(x/2)
a = np.pi**4*(1-b)
y2 = a/x**4
plt.figure(facecolor="white")
plt.plot(x,y,color="blue")
plt.plot(x,y2)
ax = plt.axes()
ax.set_facecolor("white")
ax.plot(x,y,color="blue")
ax.plot(x,y2,color='orange')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='green')
ax.tick_params(axis='y', colors='black')
plt.legend(['1','2'],labelcolor='linecolor',facecolor='white')
plt.show()

In [40]:
# Streuquerschnitte wurden integgriert. Die Funktionen wurden allerdings nicht invertiert um die Wahrscheinlichkeitsverteilung einfacher zu generieren. 
# Sie wurden nur verwendent um die Implementierung schneller zu machen
from sympy import *
init_printing()
x = Symbol('x')
b = Symbol('b')
# integrate(1/sin(x)**4*(3+cos(x)**2)**2,x)
integrate(1/sin(x/2)**4-b/sin(x/2)**2,x)


In [144]:

# cut-off-winkel
angle_cut = 0.0016

# integrierter Møllerquerschnitt
def møller_int(x):
    return (x+2*np.tan(x/2)**3/3+2*np.tan(x/2)-2/np.tan(x/2)-2/3/np.tan(x/2)**3)

import random
# integrierter Mottquerschnitt
def mott_int(y,a):
    return (2*a*np.cos(y/2)/np.sin(y/2)-4*np.cos(y/2)/3/np.sin(y/2)-2*np.cos(y/2)/3/np.sin(y/2)**3)

def mott(x,b):
    return 1/np.sin(x/2)**4-b/np.sin(x/2)**2

# zufallsgenerator Mottstreuung
def rand_mott(rand,beta):
    # integrierte Fläche (referenz)
    A_mot = mott_int(np.pi,beta)-mott_int(angle_cut,beta)
    # einhüllende hat die Form 1/x^4 aber mit verschiedenen Vorfaktoren in bestimmten Segmenten
    a1 = 1/np.sin(1/2)**4
    a2 = np.pi**4*(1-b)
    A1 = a1/3*(1/angle_cut**3-1)
    A2 = a2/3*(1-1/np.pi**4)
    rand *= (A1+A2)
    # x-Intervall wurde in verschiedene Segmente aufgeteilt, je nach segment wird für y die passende Einhüllende gewählt
    if rand <= A1:
        x = (1/angle_cut**3-3*rand/a1)**(-1/3)
        if ((random.random()*a1/x**4)<=(mott(x,beta)/A_mot*(A1+A2))):
            return x
        else:
            return -1
    else:
        x = (1-3/a2*(rand-A1))**(-1/3)
        if ((random.random()*a2/x**4)<=(mott(x,beta)/A_mot*(A1+A2))):
            return x
        else:
            return -1

alpha = 1/137
def møller(x):
    return 1/np.sin(x)**4*(3+np.cos(x)**2)**2

def møller_int(x):
    return (x+2*np.tan(x/2)**3/3+2*np.tan(x/2)-2/np.tan(x/2)-2/3/np.tan(x/2)**3)
# inverse von Stamfunktion von a/x^4 für die verschiedenen Segmente und abhängig von cut-off Winkel x0
def x4_inv(a,A,x0,y):
    return (1/x0**3-3/a*(y-A))**(-1/3)

# von der rechten Seite (pi) ausgerechnet (überflüssig)
def x4_inv_rh(a,A,x0,y):
    return (np.pi-(1/(np.pi-x0)**3-3/a*(y-A))**(-1/3))

A_mø = møller_int(np.pi-0.1)-møller_int(0.1)
mø_a1= møller(np.pi/2)*(np.pi)**4
mø_a2 = møller(1)
mø_a3 = møller(1/2)/2**4
mø_A1 = mø_a1/3*(1/angle_cut**3-2**3)
mø_A2 = mø_a2/3*(2**3-1)
mø_A3 = mø_a3/3*(1-(2/np.pi)**3)
def rand_møller(rand):
    rand *= A_mø
    if (rand<mø_A1):
        x = x4_inv(mø_a1,0,angle_cut,rand)
        if (random.random()*mø_a1/x**4 <= møller(x)/A_mø*2*(mø_A1+mø_A2+mø_A3)):
            return x
        else:
            return -1
    elif (rand>= mø_A1 and rand<(mø_A2+mø_A1)):
        x = x4_inv(mø_a2,mø_A1,0.5,rand)
        if (random.random()*mø_a2/x**4 <= møller(x)/A_mø*2*(mø_A1+mø_A2+mø_A3)):
            return x
        else:
            return -1
    elif (rand>= (mø_A2+mø_A1) and rand<(mø_A2+mø_A3+mø_A1)):
        x = x4_inv(mø_a3,mø_A1+mø_A2,1,rand)
        if (random.random()*mø_a3/x**4 <= møller(x)/A_mø*2*(mø_A1+mø_A2+mø_A3)):
            return x
        else:
            return -1
    elif (rand>= (mø_A2+mø_A1+mø_A3) and rand<(mø_A2+mø_A3*2+mø_A1)):
        x = np.pi-x4_inv(mø_a3,mø_A1+mø_A2+mø_A3,1,rand)
        if (random.random()*mø_a3/x**4 <= møller(x)/A_mø*2*(mø_A1+mø_A2+mø_A3)):
            return x
        else:
            return -1
    elif (rand>= (mø_A2+mø_A1+mø_A3*2) and rand<(mø_A2*2+mø_A3*2+mø_A1)):
        x = np.pi-x4_inv(mø_a2,mø_A1+mø_A2+2*mø_A3,0.5,rand)
        if (random.random()*mø_a2/x**4 <= møller(x)/A_mø*2*(mø_A1+mø_A2+mø_A3)):
            return x
        else:
            return -1
    elif (rand>= (mø_A2+mø_A1*2+mø_A3*2) and rand<(mø_A2*2+mø_A3*2+mø_A1*2)):
        x = np.pi-x4_inv(mø_a1,mø_A1+mø_A2*2+2*mø_A3,angle_cut,rand)
        if (random.random()*mø_a1/(np.pi-x)**4 <= møller(x)/A_mø*2*(mø_A1+mø_A2+mø_A3)):
            return x
        else:
            return -1

# lambda in der freien Weglänge Funktion wird abgeschätzt
import scipy.constants as cons
lamb= 0.9*1000/cons.u/(28)*(4*2.625+2+2*16.762)*2.8002852/10**21
lamb2=lamb/cons.c/cons.hbar/cons.eV

# Zufallsgenerator freoe Weglänge
def rand_free_wl(rand):
    return -np.log(1-rand)/lamb

print(1/cons.c/cons.hbar/cons.eV)
lamb
    

1.9742072490999566e+44


2494721137.1936636

In [16]:
# Rotationsmatrizen werden definiert (index=Drehachse (Schreibweise))
def rot_z(cos,sin):
    return np.array([[cos,-sin,0],[sin,cos,0],[0,0,1]])
def rot_y(cos,sin):
    return np.array([[cos,0,-sin],[0,1,0],[sin,0,cos]])


#### in diesem Block werden die relativistischen Größenumrechnungen definiert

In [34]:

import scipy.constants as cons
# lamb muss nicht in MeV um gerechnet werden, da proportionalitätsconstante nicht bekannt ist und man Umrechnung implizit dahinein stecken kann.
lamb = 0.9*1000/cons.u/(28)*(4*2.625+2+2*16.762)*2.8002852/10**21
# Rechnung in Mev
me = 0.511


# def vb(T):
#     return np.sqrt(1-1/gamm(T)**2)

# def p_cm(T):
#     return gamm(T)*(p0(T)-vb(T)*(T+me))

# def p2(p,theta):
#     return (2*me*(np.sqrt(p**2+me**2))*p*np.cos(theta)/(p**2+me**2-p**2*np.cos(theta)**2))

# def my_p2(p,theta,T):
#     return p**2/(2*me*T)*(p-(T+me)/p*T)/T**2*np.sqrt(np.cos(theta)**2*T**4+2*np.cos(theta)*T**2*p**2+2*p**4)

# def E_kin(p):
#     return np.sqrt(p**2+me**2)-me

# def gamm(T):
#     return (T+me)/np.sqrt(2*me**2+2*T*me)

# def angle(E,theta):
#     return np.sin(theta)/gamm(E)/(1+np.cos(theta))

# Impuls im Laborsystem
def p0(T):
    return np.sqrt((T+me)**2-me**2)

# geschwindigkeit im Laborsystem
def v0(T):
    return p0(T)/(T+me)
# Boost-Geschwindigkeit siehe Protokoll
def my_VB(T):
    v = v0(T)
    return 1/v-1/v*np.sqrt(1-v**2)

# gamma-Faktor als Funktion der Geschwindigkeit
def gam(v):
    return 1/np.sqrt(1-v**2)

# Impuls im Schwerpunksystem
def my_p1(T):
    return gam(my_VB(T))*(p0(T)-my_VB(T)*gam(v0(T))*me)

# Energie im Schwerpunktsystem (nullte komponenten des vierer Impulses)
def E1(T):
    return gam(my_VB(T))*((T+me)-my_VB(T)*p0(T))

# nach Drehung zurück ins Laborsystem transformierter impuls und zwar die komponente parallel zur ursprünglichen Geschwindigkeit
def p2x(T,theta):
    return gam(my_VB(T))*(my_p1(T)*np.cos(theta)+my_VB(T)*E1(T))

# Betrag des nach drehung zurück ins Laborsystem transformierten Impulses
def p2_mod(T,theta):
    return np.sqrt(p2x(T,theta)**2+my_p1(T)**2*np.sin(theta)**2)

# definition eines allgemeinen Einheitsvektors
def unit_v(thet,phi):
    return np.array([np.sin(thet)*np.cos(phi),np.sin(thet)*np.sin(phi),np.cos(thet)])

T=20
# print(E_kin(p2(p0(T),0.12)))
print(T-1/lamb/T)

67.37215153121308
19.99999999997996


In [159]:
angle_cut = 0.0008
# Elektronenzahl
numb_e =20
# Liste mit späteren Trajektorien
trajectories = []
lamb= 0.9*1000/cons.u/(28)*(4*2.625+2+2*16.762)*2.8002852/10**21

# pro Elektorn gezählte Iterationen
count_store = np.zeros(numb_e+1)
count =0
# Abrruchkrierium
Tmin = 0.511
for iter in range(numb_e):
    trajectories.append(np.zeros((1000000,3)))
    direction = np.array([0,0,1])
    T = 20
    count_store[iter] =count
    count =0
    # archiv der freien Weglängen
    x_store = np.zeros(1000000)
    while (T>Tmin):
        count +=1
        rando = random.random()
        # Ziehung freie Weglänge
        x = rand_free_wl(rando)
        # Abbruch falls freie Weglänge potential der Energie übersteigen würde
        if x>T**2:
            print(x)
            break
        # CSDA 
        T=T-x/T*10**6
        x_store[count]=x
        # Aufzeichnung der Elektronenbahn
        trajectories[iter][count,:]=trajectories[iter][count-1,:]+direction*x

        thet = -1
        # Wahl ob Mott oder Møller Streuung
        if (random.random()<1/2):
            # Ziehung des Winkels nach Møller
            while (thet==-1):
                thet = rand_mott(random.random(),v0(T))
            # Energieverlust durch Stoß wenn Winkel >0.2
            if thet>=0.2:
                T= np.sqrt(p2_mod(T,thet)**2+me**2)-me
        # Ziehung des Winkels nach Mott
        else:
            while (thet==-1):
                thet = rand_møller(random.random())
        # Berechnung von cos/sin von theta/phi
        PL_mod = np.sqrt(direction[0]**2+direction[1]**2+direction[2]**2)
        rho_L = np.sqrt(direction[0]**2+direction[1]**2)
        if PL_mod==0:
            cos_thet = 1
            sin_thet = 0
            cos_phi = 1
            sin_phi = 0
        elif rho_L ==0:
            cos_thet = 1
            sin_thet = 0
            cos_phi = 1
            sin_phi = 0
        else:
            cos_thet = direction[2]/PL_mod
            sin_thet = rho_L/PL_mod
            cos_phi = direction[0]/rho_L
            sin_phi = direction[1]/rho_L
        # Durchführung der gezogenen Richungsänderung
        direction = np.matmul(rot_z(cos_phi,sin_phi),np.matmul(rot_y(cos_thet,-sin_thet),unit_v(thet,random.random()*2*np.pi)))
        # Abbruch nach max iterationen
        if count==1000000-1:
            print(T)
            print(x_store)
            break
        elif (T<=20==False):
            print(T)
            break
        
count_store[numb_e]=count


In [128]:
trajectories_save, count_store_save = trajectories, count_store

In [153]:
trajectories_save2, count_store_save2 = trajectories, count_store

In [156]:
trajectories, count_store= trajectories_save2, count_store_save2

In [157]:
# 2D Plot der Elektronenbahnen
plt.figure(facecolor="white")

ax = plt.axes()
ax.set_facecolor("white")
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.tick_params(axis='x', colors='black')
ax.ticklabel_format(style='scientific',scilimits=(-3,3))
ax.tick_params(axis='y', colors='black')
ax.set_xlabel('y-Achse')
ax.set_ylabel('z-Achse')
for iter in range(numb_e):
    count = int(count_store[iter+1])
    
    points = trajectories[iter][:count,:]
    ax.plot(points[:,1],points[:,2])
plt.show()

In [173]:
# 3D Plot

# import matplotlib.pyplot as plt
# %matplotlib notebook

# Ausgabe der benötigten Iterationen
print(count_store)

# plt.legend(['1','2'],labelcolor='linecolor',facecolor='white')
ax = plt.figure(facecolor='white').add_subplot(projection='3d')
ax.spines['bottom'].set_color('black')
ax.spines['left'].set_color('black')
ax.xaxis.label.set_color('black')
ax.yaxis.label.set_color('black')
ax.zaxis.label.set_color('black')
ax.tick_params(axis='y', colors='black',labelrotation=40)
ax.tick_params(axis='x', colors='black')#,labelrotation=40)

ax.tick_params(axis='z', colors='black')
ax.set_facecolor("white")
# ax.ticklabel_format(axis='y',style='scientific',scilimits=(-3,3))
ax.view_init(20,25,0)
for iter in range(numb_e):
    count = int(count_store[iter+1])
    
    points = trajectories[iter][:count,:]
    ax.plot(points[:,0],points[:,1],points[:,2])
plt.show()


[     0. 497344. 498227. 497034. 499868. 498261. 498776. 498547. 499336.
 498836. 498669. 496841. 497945. 497267. 499388. 497744. 497712. 499191.
 498874. 499297. 496993.]
